In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import torch
import torch.nn.functional as F
from torch.nn.attention.flex_attention import flex_attention, create_block_mask
import torch

In [3]:
head_num = 16
dim = 128
seq_len = 100
chunk_size = 5
batch_size = 1

In [4]:
q = torch.randn(batch_size, head_num, seq_len, dim).cuda().to(torch.bfloat16)
k = torch.randn(batch_size, head_num, seq_len, dim).cuda().to(torch.bfloat16)
v = torch.randn(batch_size, head_num, seq_len, dim).cuda().to(torch.bfloat16)

In [5]:
def noop(score, b, h, q_idx, kv_idx):
    return score

In [15]:
flex_attention

<function torch.nn.attention.flex_attention.flex_attention(query: torch.Tensor, key: torch.Tensor, value: torch.Tensor, score_mod: Optional[Callable[[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor], torch.Tensor]] = None, block_mask: Optional[torch.nn.attention.flex_attention.BlockMask] = None, scale: Optional[float] = None, enable_gqa: bool = False, return_lse: bool = False, kernel_options: Optional[Dict[str, Any]] = None) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]>

In [20]:
actual_out, actual_lse = flex_attention(q, k, v, block_mask = None, return_lse=True, )

In [21]:
actual_out

tensor([[[[-0.0265, -0.1709, -0.0206,  ...,  0.3535,  0.0334, -0.0204],
          [ 0.2480, -0.2061, -0.0669,  ...,  0.2793, -0.3281,  0.1250],
          [ 0.0347, -0.2520, -0.1055,  ...,  0.0444,  0.0229,  0.1279],
          ...,
          [-0.0304, -0.1279,  0.1128,  ...,  0.1875, -0.0535, -0.0981],
          [-0.1250, -0.1147, -0.1084,  ...,  0.2520, -0.0491,  0.1641],
          [ 0.1309, -0.2432, -0.1641,  ...,  0.2500,  0.0305, -0.0302]],

         [[-0.1357, -0.3730,  0.1138,  ..., -0.0289,  0.2119,  0.3281],
          [-0.0562,  0.0796,  0.0913,  ..., -0.0449,  0.0175,  0.0786],
          [-0.3164,  0.2207,  0.3027,  ..., -0.1387,  0.0449,  0.2285],
          ...,
          [-0.1216, -0.0277,  0.1650,  ..., -0.0014,  0.0197,  0.2285],
          [-0.1147,  0.0266,  0.0173,  ..., -0.1147,  0.0684,  0.1924],
          [-0.0400,  0.1074,  0.0299,  ...,  0.0791,  0.0923,  0.1260]],

         [[ 0.1377, -0.0610, -0.0864,  ..., -0.2295,  0.2031, -0.0879],
          [ 0.0903,  0.0105,  

In [7]:
q_chunks = q.chunk(chunk_size, dim = 2)
k_chunks = k.chunk(chunk_size, dim = 2)
v_chunks = v.chunk(chunk_size, dim = 2)

In [8]:
Q_block = q_chunks[0]

In [9]:
out, lse = None, None

def merge_attention(a, lse_a, b, lse_b):
    max_lse = torch.maximum(lse_a, lse_b)
    lse_a = torch.exp(lse_a - max_lse)
    lse_b = torch.exp(lse_b - max_lse)
    out = ((a * lse_a[..., None] + b * lse_b[..., None]) / (lse_a + lse_b)[..., None])
    return out, torch.log(lse_a + lse_b) + max_lse

for i in range(len(k_chunks)):
    block_out, block_lse = flex_attention(Q_block, k_chunks[i], v_chunks[i], score_mod=noop, return_lse=True)
    if out is None:
        out = block_out
        lse = block_lse
    else:
        out, lse = merge_attention(out, lse, block_out, block_lse)

In [11]:
actual_out[...,:out.shape[2], :].argmax(-1)

tensor([[[ 36,  67,  38,  36,  98,  64,  38,  25,  38,  38,  26,  24, 125, 125,
           67, 125,  55, 125,  38,  38],
         [ 69,   5,  47,  47,  25,  63,   5,   5,  76,  40,  51,   5,   5,  76,
            5,  76,   5,  85,  70,  25],
         [ 14,  58, 115,  37,  58,  58, 115, 115,  87,  77, 115,  21,  28,  14,
           96,  96,  14,  32,  14,  53],
         [ 52,  31,  11, 105, 122,  54, 105, 122,  45, 105,  77, 107, 122,  21,
           66,  66,  66,  76, 122, 107],
         [ 92,  40,   5,  83,  74,  82,  83,  94, 115,  40,  65, 108,   0,  42,
           65,  19, 106, 109,  96, 108],
         [ 88,  94, 106, 113,  35,  94, 106,  48,  48,  69,  35,  48,   4, 124,
           88,  94,  61,  48,  60,  87],
         [ 24,  82,  13,  45,  42,   8,  74,  42,  45,  42,  58,  99,  69,  24,
           72,  33,  31,  43, 107,  31],
         [  0,  28, 123,  57,  58,  70,  72, 121,  72, 123,  78, 121,  71,  51,
           72,  73,  93,  97,  72,  72],
         [ 10,  95,  95,  79,  9

In [12]:
out.argmax(-1)

tensor([[[ 36,  67,  38,  36,  98,  64,  38,  25,  38,  38,  26,  24, 125, 125,
           67, 125,  55, 125,  38,  38],
         [ 69,   5,  47,  47,  25,  63,   5,   5,  76,  40,  51,   5,   5,  76,
            5,  76,   5,  85,  70,  25],
         [ 14,  58, 115,  37,  58,  58, 115, 115,  87,  77, 115,  21,  28,  14,
           96,  96,  14,  32,  14,  53],
         [ 52,  31,  11, 105, 122,  54, 105, 122,  45, 105,  77, 107, 122,  21,
           66,  66,  66,  76, 122, 107],
         [ 92,  40,   5,  96,  74,  82,  83,  94, 115,  40,  65, 108,   0,  42,
           65,  19, 106, 109,  96, 108],
         [ 88,  94, 106, 113,  35,  94, 106,  48,  48,  69,  35,  48,   4, 124,
           88,  94,  61,  48,  60,  87],
         [ 24,  82,  13,  45,  42,   8,  74,  42,  45,  42,  58, 116,  69,  24,
           72,  33,  31,  43, 107,  31],
         [  0,  28, 123,  57,  58,  70,  72, 121,  72, 123,  78, 121,  71,  51,
           72,  73,  93,  97,  72,  72],
         [ 10,  95,  95,  79,  9

In [13]:
out - actual_out[...,:out.shape[2], :]

tensor([[[[-1.1434e-04, -9.2879e-05,  6.1715e-04,  ...,  1.7706e-04,
           -9.9272e-05, -9.2408e-05],
          [-3.3399e-04, -4.9867e-04,  7.5468e-04,  ..., -7.2905e-04,
           -1.3507e-03,  9.8483e-04],
          [ 3.2492e-05,  1.3351e-05, -3.0652e-04,  ..., -5.8907e-04,
            3.0853e-05,  4.2535e-04],
          ...,
          [ 3.7903e-04, -2.2210e-04,  3.0227e-04,  ..., -1.1069e-04,
            4.5701e-04,  8.5687e-05],
          [-7.1747e-05,  8.6533e-04, -8.2013e-04,  ..., -8.9389e-04,
            1.1049e-03,  6.4605e-04],
          [ 6.9747e-04,  2.2663e-04,  1.0055e-03,  ..., -1.0483e-04,
            4.3447e-04, -5.3114e-04]],

         [[ 1.1492e-04, -1.1820e-03,  2.5764e-05,  ..., -3.5966e-04,
            4.6501e-04,  1.7661e-04],
          [-6.0832e-04, -8.4080e-05,  3.9133e-04,  ...,  3.8039e-04,
            4.3546e-04,  6.7694e-04],
          [ 1.6277e-03,  1.2225e-03,  5.2315e-04,  ...,  1.1359e-04,
           -2.0671e-04, -6.5893e-05],
          ...,
     